# PyMem3DG Tutorial 4 - Visualize Output
`Cuncheng Zhu, Christopher T. Lee`

Mem3DG output the trajectory data using a combination of NetCDF and $\texttt{.ply}$ file. There are many ways to read the data. This tutorial provides a minimal set of functions from Python NetCDF and PyMem3DG to extract the data and visualize using Polyscope. 

In [ ]:
import pymem3dg as dg
import pymem3dg.visual as dg_vis
import pymem3dg.read as dg_read
import pymem3dg.read.netcdf as dg_nc

import pymem3dg.util as dg_util
import pymem3dg.boilerplate as dgb
from functools import partial
from pathlib import Path


import numpy as np
import polyscope as ps
import netCDF4 as nc
import matplotlib.pyplot as plt

## NetCDF
Python netCDF4 library has conveient functions to extract a $\texttt{.nc}$ file. Here we provide an example of reading the file using data generated from tutorial 1.

In [ ]:
trajFile = "output/tutorial1/biconcave/traj.nc"

p = dg.Parameters()
p.bending.Kbc = 8.22e-5
p.tension.form = partial(
    dgb.preferredAreaSurfaceTensionModel, modulus=0.1, preferredArea=12.4866
)
p.osmotic.form = partial(
    dgb.preferredVolumeOsmoticPressureModel,
    preferredVolume=0.7 * 3.14 * 4 / 3,
    reservoirVolume=0,
    strength=0.02,
)


frameLim = (0, dg_nc.sizeOf(trajFile))
frameNum = frameLim[1] - frameLim[0]
time = np.zeros(frameNum)
kineticEnergy = np.zeros(frameNum)
potentialEnergy = np.zeros(frameNum)
externalWork = np.zeros(frameNum)
totalEnergy = np.zeros(frameNum)
volume = np.zeros(frameNum)


for frame in range(frameNum):
    geo = dg.Geometry(trajFile, frame)
    system = dg.System(
        geometry=geo, trajFile=trajFile, startingFrame=frame, parameters=p
    )
    system.initialize()
    time[frame] = system.time
    volume[frame] = geo.getVolume()
    system.computeTotalEnergy()
    energy = system.getEnergy()
    kineticEnergy[frame] = energy.kineticEnergy
    potentialEnergy[frame] = energy.potentialEnergy
    if frame != 0:
        externalWork[frame] = externalWork[frame - 1] + system.computeIntegratedPower(
            time[frame] - time[frame - 1]
        )
totalEnergy = potentialEnergy + kineticEnergy - externalWork
reducedVolume = volume / (3.14 * 4 / 3)

# plotting
fig, ax1 = plt.subplots()
color = "tab:red"
ax1.set_xlabel("time")
ax1.set_ylabel("energy", color=color)
ax1.plot(time, totalEnergy, color=color)
ax1.tick_params(axis="y", labelcolor=color)

ax2 = ax1.twinx()
color = "tab:blue"
ax2.set_ylabel("reduced volume", color=color)
ax2.plot(time, reducedVolume, color=color)
ax2.tick_params(axis="y", labelcolor=color)

fig.tight_layout()
plt.show()


Unlike the $\texttt{.ply}$ file that we will cover below, NetCDF file covers the full trajectory. Therefore, the first index of the resultant matrix is always the frame index.

## $\texttt{.ply}$
Mem3DG provide some wrapper functions of a C++ library Happly to help extracting data from $\texttt{.ply}$ file.

In [ ]:
ply = 'output/tutorial4/rbc.ply'
face, vertex = dg_read.mesh.getFaceAndVertexMatrix(ply)
print(dg_read.mesh.getDataElementName(ply))
print(dg_read.mesh.getDataPropertyName(ply, 'vertex'))
H = dg_read.mesh.getData(ply, 'vertex', 'mean_curvature')
Fb = dg_read.mesh.getData(ply, 'vertex', 'bending_force')

## Visualize using Polyscope
Details and functionalities of Polyscope is located at https://polyscope.run/py/

In [ ]:
ps.init()
ps_mesh = ps.register_surface_mesh("RBC", vertex, face)
ps_mesh.add_scalar_quantity("mean_curvature", H, enabled=True)
ps_mesh.add_scalar_quantity("bending_force", Fb, enabled=True, vminmax=(-1e-5, 1e-5))
ps.set_up_dir("z_up")
ps.show()